In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Helper function
def findMinMax(data):
  q1 = data.quantile(0.25)
  q3 = data.quantile(0.75)
  min = q1 - 1.5 * (q3 - q1)
  max = q3 + 1.5 * (q3 - q1)
  return min, max

# Function to help us visualize the outliers. Produce boxplot and print out outliers
def getOutliers(data, features):
  for i, feature in enumerate(features, 1):
    plt.subplot(1,len(features),i)
    data[[feature]].boxplot()
    
    min, max = findMinMax(data[feature])
    outliers_lower = data[feature] < min
    outliers_upper = data[feature] > max
    
    if outliers_lower.any():
      print(feature, "- Lower outliers:\n", data.loc[outliers_lower, feature])
    if outliers_upper.any():
      print(feature, "- Upper outliers:\n", data.loc[outliers_upper, feature])

  plt.show()
  

# return az set of data with outliers removed
def removeOutliers(data, features):
  removeIdx = pd.Series([False] * len(data))
  for i, feature in enumerate(features):
    min, max = findMinMax(data[feature])
    outliers_lower = data[feature] < min
    outliers_upper = data[feature] > max
    
    removeIdx = removeIdx | outliers_lower | outliers_upper

  return data.loc[~removeIdx]
  


# Testing code
data = pd.DataFrame(pd.read_csv('./SeoulBikeData.csv'))
features = ['Rented Bike Count', 'Wind speed (m/s)']

getOutliers(data, features)

newData = removeOutliers(data, features)
getOutliers(newData, features)

In [ ]:
from sklearn.preprocessing import StandardScaler

data = pd.DataFrame(pd.read_csv('./SeoulBikeData.csv'))

# Remove rows with non-functioning day / no bike rented
functioningDay = data['Functioning Day'] == 'Yes'
data = data.loc[functioningDay]

# Dropping some features:
# Date: can't process and we already have the holiday feature
# Dew temp: not relevant
# Functioning day: already process
data = data.drop(columns=['Date', 'Dew point temperature', 'Functioning Day'])


# One Hot Encode categorical features
# Hour should be categorical too. Not sure how to handle it yet
data = pd.get_dummies(data, columns=['Seasons'], dtype=int)
data = pd.get_dummies(data, columns=['Holiday'], dtype=int, drop_first=True)


getOutliers(data, ['Rented Bike Count', 'Temperature', 'Humidity', 'Wind speed', 'Visibility', 'Solar Radiation', 'Rainfall', 'Snowfall'])

# It seems like every rainy or snowy days are counted as outliers because the weather is normal most of the time.
# Therefore, not going to remove outliers for Rainfall and Snowfall

# A lot of outliers for Solar Radiation. We can test this out with our models. For now, not removing outliers for this one

# Around 150 outliers for Rented Bike Count and Wind Speed. Remove outliers for now

data = removeOutliers(data, ['Rented Bike Count', 'Temperature', 'Humidity', 'Wind speed', 'Visibility'])
data = data.reset_index(drop=True)
print(data)


# Splitting data between categorical and numericals set for standardization
categoricalFeatures = ['Hour', 'Seasons_Autumn', 'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter', 'Holiday_No Holiday']
numericalFeatures = ['Rented Bike Count', 'Temperature', 'Humidity', 'Wind speed', 'Visibility', 'Solar Radiation', 'Rainfall', 'Snowfall']
categoricalValues = data[categoricalFeatures]
standardizedData = data.drop(columns=categoricalFeatures)

scaler = StandardScaler()
scaler.fit(standardizedData)
standardizedData = scaler.transform(standardizedData)
standardizedData = pd.DataFrame(standardizedData)
standardizedData.columns = numericalFeatures

standardizedData = pd.concat([standardizedData, categoricalValues], axis=1)
print(standardizedData)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# not sure if this function is totally necessary because it runs without it
def bgd(X, y, learning_rate=0.01, batch_size=32, reg_para=0.01, epochs=100, clip_value=5.0):
    
    n_samples, n_features = X.shape
    
    # Initialize weights with small random values
    weights = np.random.randn(n_features) * 0.01
    bias = 0
    loss_history = []
    
    # Calculate number of batches
    n_batches = int(np.ceil(n_samples / batch_size))
    
    for epoch in range(epochs):
        indices = np.random.permutation(n_samples)
        X_shuffled = X[indices]
        y_shuffled = y[indices]
        
        epoch_loss = 0
        
        for batch in range(n_batches):
            start_idx = batch * batch_size
            end_idx = min((batch + 1) * batch_size, n_samples)
            X_batch = X_shuffled[start_idx:end_idx]
            y_batch = y_shuffled[start_idx:end_idx]
            batch_size_actual = end_idx - start_idx
            
            # Forward pass with numerical stability
            y_pred = np.clip(np.dot(X_batch, weights) + bias, -1e15, 1e15)
            
            # Calculate gradients with numerical stability
            errors = y_pred - y_batch
            dw = (1/batch_size_actual) * (np.dot(X_batch.T, errors) + reg_para * weights)
            db = np.mean(errors)
            
            # Gradient clipping
            dw = np.clip(dw, -clip_value, clip_value)
            db = np.clip(db, -clip_value, clip_value)
            
            # Update weights and bias with smaller learning rate
            weights -= learning_rate * dw
            bias -= learning_rate * db
            
            # Calculate loss with numerical stability
            squared_errors = np.clip(errors ** 2, 0, 1e15)
            batch_loss = np.mean(squared_errors) + (reg_para/(2*batch_size_actual)) * np.sum(np.clip(weights ** 2, 0, 1e15))
            epoch_loss += batch_loss
            
            # Check for NaN values
            if np.isnan(batch_loss) or np.isnan(weights).any():
                print(f"NaN detected at epoch {epoch}, batch {batch}. Stopping training.")
                return weights, bias, loss_history
        
        epoch_loss /= n_batches
        loss_history.append(epoch_loss)
        
        # Early stopping if loss is very small
        if epoch > 0 and abs(loss_history[-1] - loss_history[-2]) < 1e-7:
            break
            
    return weights, bias, loss_history


def predict_batch_gd(X, weights, bias):
    # prevent overflow
    return np.clip(np.dot(X, weights) + bias, -1e15, 1e15)

# batch gradient descent with l2 regularization
def grid_search_bgd(X, y, folds=10, degrees=[1, 2], 
                   batch_sizes=[32, 64],
                   learning_rates=[0.00001, 0.0001], 
                   reg_parameters=[0.00001, 0.0001]):
    
    # store best results
    best_mse = float('inf')
    best_params = None
    best_weights = None
    best_bias = None

    # need to convert to numpy arrays for compatibility w sklearn
    X_np = X.to_numpy() if hasattr(X, 'to_numpy') else X
    y_np = y.to_numpy() if hasattr(y, 'to_numpy') else y
    
    # scale y to prevent overflow
    y_scale = np.max(np.abs(y_np))
    y_np = y_np / y_scale
    
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    results = []
    
    for degree in degrees:
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        X_poly = poly.fit_transform(X_np)
        
        # scale X to prevent overflow
        X_scale = np.max(np.abs(X_poly))
        X_poly = X_poly / X_scale
        
        for batch_size in batch_sizes:
            for learning_rate in learning_rates:
                for reg_para in reg_parameters:
                    
                    fold_mses = []
                    
                    for train_index, val_index in kf.split(X_poly):
                        X_train, X_val = X_poly[train_index], X_poly[val_index]
                        y_train, y_val = y_np[train_index], y_np[val_index]
                        
                        try:
                            weights, bias, loss_history = bgd(
                                X_train, y_train,
                                learning_rate=learning_rate,
                                batch_size=batch_size,
                                reg_para=reg_para
                            )
                            
                            y_pred = predict_batch_gd(X_val, weights, bias)
                            
                            # Scale predictions back
                            y_pred = y_pred * y_scale
                            y_val_original = y_val * y_scale
                            
                            mse = mean_squared_error(y_val_original, y_pred)
                            fold_mses.append(mse)
                        except Exception as e:
                            print(f"Error in fold: {str(e)}")
                            continue
                    
                    if fold_mses:  # only if we have valid results
                        avg_mse = np.mean(fold_mses)
                        
                        results.append({
                            'degree': degree,
                            'batch_size': batch_size,
                            'learning_rate': learning_rate,
                            'reg_para': reg_para,
                            'mse': avg_mse
                        })
                        
                        if avg_mse < best_mse:
                            best_mse = avg_mse
                            best_params = {
                                'degree': degree,
                                'batch_size': batch_size,
                                'learning_rate': learning_rate,
                                'reg_para': reg_para
                            }
                            best_weights = weights * X_scale
                            best_bias = bias * y_scale
    
    if best_params is None:
        raise ValueError("No valid models found. Adjust hyperparameters.")
    
    print("\nBest parameters:")
    for param, value in best_params.items():
        print(f"{param}: {value}")
    print(f"Best Mean Squared Error: {best_mse:.4f}")
    
    return best_weights, best_bias, best_params, results

def plot_training_history(loss_history):
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history)
    plt.title('Training Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.show()



# running code
# Get feature matrix X and target variable y
X = standardizedData.drop('Rented Bike Count', axis=1)
y = standardizedData['Rented Bike Count']

print("Shape of data:")
print("X shape:", X.shape)
print("y shape:", y.shape)

# Define the parameter grid 
param_grid = {
    'degrees': [1, 2],  # Start with linear model
    'batch_sizes': [16, 32, 64],  
    'learning_rates': [0.00001, 0.0001, 0.001],  # Much smaller learning rates
    'reg_parameters': [0.00001, 0.0001, 0.001]  # Smaller regularization parameters
}

# Perform grid search with cross validation
try:
    best_weights, best_bias, best_params, results = grid_search_bgd(
        X, y,
        folds=5,  # Reduced folds for faster testing
        **param_grid
    )

    # Print results summary
    print("\nTraining Results Summary:")
    print("-------------------------")
    for param, value in best_params.items():
        print(f"{param}: {value}")

    # Create predictions using best model
    poly = PolynomialFeatures(degree=best_params['degree'], include_bias=False)
    X_poly = poly.fit_transform(X)
    predictions = predict_batch_gd(X_poly, best_weights, best_bias)

    # Calculate and print final MSE
    final_mse = mean_squared_error(y, predictions)
    print(f"\nFinal MSE on full dataset: {final_mse:.4f}")

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.scatter(y, predictions, alpha=0.5)
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
    plt.xlabel('Actual Bike Rentals')
    plt.ylabel('Predicted Bike Rentals')
    plt.title('Actual vs Predicted Bike Rentals')
    plt.grid(True)
    plt.show()

except Exception as e:
    print(f"An error occurred: {str(e)}")

"""
numpy .clip: https://numpy.org/doc/2.1/reference/generated/numpy.clip.html
numpy .dot: https://numpy.org/doc/2.1/reference/generated/numpy.dot.html

"""